In [1]:
SESSION_TOKEN="53616c7465645f5f41f7a7b1200308fca2d2ba72fda79b5433224b269226e2d01e5c52af3b54e07c8c1765c603b2633803d7cac35090c984331d1da0d20d6274"

In [3]:
! pip install requests python-dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
import requests
from pathlib import Path
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

def fetch_input(day, year, session_token, save_dir="inputs"):
    """
    Fetches input for a specific Advent of Code day and saves it locally.

    :param day: The day of the Advent of Code (1-25).
    :param year: The Advent of Code year.
    :param save_dir: Directory to save the input file.
    :return: Path to the input file.
    """
    # session_token = os.getenv("SESSION_TOKEN")
    if not session_token:
        raise ValueError("Session token not found in environment variables.")

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    headers = {
        "Cookie": f"session={session_token}",
        "User-Agent": "https://github.com/sai-pendyala/Advent-of-Code-2024",
    }

    # Create the directory if it doesn't exist
    Path(save_dir).mkdir(exist_ok=True)
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    # Fetch input
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        with open(input_file, "w") as f:
            f.write(response.text.strip())
        print(f"Input for Day {day} saved to {input_file}")
        return input_file
    elif response.status_code == 404:
        print("Puzzle input is not available yet. Check after the problem is released.")
    else:
        print(f"Failed to fetch input: {response.status_code}")
    return None

In [5]:
def read_input(day, save_dir="inputs"):
    """
    Reads the content of the input file for a specific day of Advent of Code.

    :param day: The day of the Advent of Code (1-25).
    :param save_dir: Directory where the input file is saved (default is 'inputs').
    :return: Content of the input file as a string.
    """
    input_file = Path(save_dir) / f"day{day:02}_input.txt"

    if input_file.exists():
        with open(input_file, "r") as file:
            content = file.read().strip()  # Read and strip any surrounding whitespace
        print(f"Content of Day {day} input:\n{content}")
        return content
    else:
        print(f"Input file for Day {day} does not exist. Please fetch the input first.")
        return None

In [7]:
# Daily Workflow
from pathlib import Path
from datetime import datetime

# Dynamic Day and Year
now = datetime.utcnow()
DAY = now.day
YEAR = now.year

# Fetch input
fetch_input(DAY, YEAR, SESSION_TOKEN)

# Read input
input_data = read_input(DAY)

/tmp/ipykernel_21006/1625487702.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


Input for Day 2 saved to inputs/day02_input.txt
Content of Day 2 input:
1 4 5 8 11 12 9
7 8 9 10 12 15 17 17
17 20 23 25 27 31
55 57 58 61 63 64 70
39 42 45 43 44
84 85 86 87 90 89 86
33 34 35 36 35 37 38 38
8 9 11 8 10 11 13 17
34 35 37 39 38 40 45
47 50 50 51 53 54
54 55 58 58 59 56
95 98 99 99 99
53 54 54 57 61
65 68 71 71 73 78
19 20 23 27 28 30 33 36
78 79 81 85 83
24 25 27 29 30 32 36 36
78 79 80 83 86 90 93 97
30 31 35 38 40 42 49
61 63 66 71 74 75
77 80 82 83 89 87
58 61 68 70 71 74 75 75
20 23 24 25 28 35 36 40
25 28 35 36 38 40 46
47 46 49 52 55
78 76 78 79 81 78
14 13 15 17 19 21 21
77 74 77 79 80 84
91 89 91 93 94 99
65 64 63 65 66 67 70
49 48 51 53 56 54 52
29 28 30 33 31 34 34
70 69 71 70 74
41 38 39 41 43 41 46
88 85 88 90 90 92
37 34 36 37 37 39 42 40
57 55 58 61 61 61
46 44 46 46 50
78 75 77 78 79 79 82 89
86 83 86 87 91 92 95
17 14 16 17 21 18
34 32 36 39 39
60 59 62 66 68 70 73 77
59 58 60 61 63 65 69 75
41 40 42 45 46 49 55 58
4 3 4 5 8 15 16 14
5 4 10 11 11
15 14 1

# Day 1

In [7]:
lefts = []
rights = []
for line in input_data.split("\n"):
  left, right = line.strip().split()
  lefts.append(int(left))
  rights.append(int(right))

In [8]:
def get_total_distance(lefts, rights):
  total_distance = 0
  for left, right in zip(lefts, rights):
    total_distance += abs(left - right)
  return total_distance

get_total_distance(sorted(lefts), sorted(rights))

1889772

In [9]:
def get_similarity_score(lefts, rights):
  from collections import Counter
  similarity = 0
  right_counts = Counter(rights)
  for left in lefts:
    if left in right_counts:
      similarity += left * right_counts[left]
  return similarity

get_similarity_score(lefts, rights)

23228917

# Day 2

In [17]:
def is_safe(report):
    sign = 1
    if report[1] < report[0]:
        sign = -1

    for i in range(1, len(report)):
        diff = report[i] - report[i - 1]
        if sign * diff < 0 or not 1 <= abs(diff) <= 3:
            return False  
    return True

In [19]:
total_safe = 0
for line in input_data.split("\n"):
    check = is_safe(list(map(int, line.split())))
    total_safe += check

total_safe

549

In [24]:
total_safe2 = 0
for line in input_data.split("\n"):
    report = list(map(int, line.split()))
    for i in range(len(report)):
        first_half = report[:i] if i > 0 else []
        second_half = report[i + 1:] if i < len(report) - 1 else []
        check = is_safe(first_half + second_half)
        if check:
            total_safe2 += 1
            break

total_safe2

589